# Phase 3 Weighted Bagging

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

import os
import re
import csv
import codecs
import gensim
import itertools
import numpy as np
import pandas as pd
import operator
import sys

from nltk import ngrams
from collections import Counter
from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.config import dataset_config
from iwillwin.data_utils.feature_engineering import FeatureCreator

from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize

import seaborn as sns
%matplotlib inline

import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

import os
import re
import csv
import codecs
import numpy as np
import pandas as pd
import operator
from os import listdir
from os.path import isfile, join

########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
np.random.seed(1337)

import pandas as pd
import operator
import sys

from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.config import dataset_config
from keras.utils import to_categorical

C:\Users\zake7\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
C:\Users\zake7\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
NB_WORDS, MAX_SEQUENCE_LENGTH = 50000, 50
data_transformer = DataTransformer(max_num_words=NB_WORDS, max_sequence_length=MAX_SEQUENCE_LENGTH, char_level=False,
                                   normalization=True, features_processed=True)
trains_nns, tests_nns, labels = data_transformer.prepare_data(dual=False)
print("Number of unique words", len(data_transformer.tokenizer.index_docs))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\zake7\AppData\Local\Temp\jieba.cache
Loading model cost 0.848 seconds.
Prefix dict has been built succesfully.


[DataHelper] Apply normalization on value-type columns


C:\Users\zake7\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Doing preprocessing...
Transforming words to indices...
Shape of data tensor: (320552, 50) (320552, 50)
Shape of label tensor: (320552,)
Preprocessed.
Number of unique words 83265


In [3]:
trains_meta = trains_nns[2]
tests_meta = tests_nns[2]

In [4]:
train_df = pd.read_csv('../data/dataset/train.csv')
test_df = pd.read_csv('../data/dataset/test.csv')

In [37]:
rumor_words = ['辟谣', '谣言', '勿传', '假的']

def is_rumor(text):
    if type(text) != str:
        print(text, type(text))
        return 0
    for rumor_word in rumor_words:
        if rumor_word in text:
            return 1
    return 0

def has_split_symbol(text):
    if type(text) != str:
        return 0
    if '|' in text:
        return 1
    return 0

for df in [train_df, test_df]:
    df['has_|'] = df['title2_zh'].apply(has_split_symbol)
    df['has_rumor_words'] = df['title2_zh'].apply(is_rumor)

nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>


In [38]:
train_has_rumor = train_df.has_rumor_words.values
test_has_rumor = test_df.has_rumor_words.values

trick_trains_features = np.concatenate((trains_nns[2], train_has_rumor.reshape((-1, 1))), axis=1)
trick_tests_features = np.concatenate((tests_nns[2], test_has_rumor.reshape((-1, 1))), axis=1)

In [39]:
oof_file_names = sorted([f for f in listdir('../data/pseudo/oofs/') if isfile(join('../data/pseudo/oofs/', f)) and f != '.gitkeep'])
preds_file_names = [name.replace('-Train', '') for name in oof_file_names]

oofs = []
preds = []
for name in oof_file_names:
    oofs.append(pd.read_csv('../data/pseudo/oofs/' + name))
for name in preds_file_names:
    preds.append(pd.read_csv('../data/pseudo/output/' + name))    

In [40]:
for i, name in enumerate(oof_file_names):
    print(i, name)
    
trains = pd.DataFrame()
tests = pd.DataFrame()

for i in range(len(oof_file_names)):
    for label_type in ['agreed', 'disagreed', 'unrelated']:
        trains['oofs_{}_{}'.format(i, label_type)] = oofs[i][label_type].values
        tests['oofs_pred{}_{}'.format(i, label_type)] = preds[i][label_type].values

0 3Embedding-DecomposalbeAttention-NoMeta-ClassWeighted-NoEM-Train-L0.857551-NB5000.csv
1 P3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM-Train-L0.297362-NB5000.csv
2 P3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM-Train-L0.292235-NB5000.csv
3 P3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM-Train-L0.283131-NB5000.csv
4 PS3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM-Train-L0.853793-NB5000.csv
5 PS3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM-Train-L0.853764-NB5000.csv
6 PS3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM-Train-L0.863459-NB5000.csv
7 PSWordSGNS-DAttn-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.861867-NB100000.csv
8 PSWordSGNS-DenseCNN5Layers-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.851927-NB100000.csv
9 PSWordSGNS-DenseCNN5Layers-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.855071-NB100000.csv
10 PSWordSGNS-ESIM-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.860951-NB100000.csv
11 PSWord

In [41]:
oof_file_names

['3Embedding-DecomposalbeAttention-NoMeta-ClassWeighted-NoEM-Train-L0.857551-NB5000.csv',
 'P3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM-Train-L0.297362-NB5000.csv',
 'P3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM-Train-L0.292235-NB5000.csv',
 'P3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM-Train-L0.283131-NB5000.csv',
 'PS3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM-Train-L0.853793-NB5000.csv',
 'PS3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM-Train-L0.853764-NB5000.csv',
 'PS3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM-Train-L0.863459-NB5000.csv',
 'PSWordSGNS-DAttn-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.861867-NB100000.csv',
 'PSWordSGNS-DenseCNN5Layers-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.851927-NB100000.csv',
 'PSWordSGNS-DenseCNN5Layers-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.855071-NB100000.csv',
 'PSWordSGNS-ESIM-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.860951-NB1

In [42]:
unrelated = pd.DataFrame()
agreeds = pd.DataFrame()
disagreeds = pd.DataFrame()

#check_oofs = True
check_oofs = False


if check_oofs:
    for i, oof in enumerate(oofs):
        agreeds['oofs_agreed_{}'.format(i)] = oofs[i]['agreed'].values
        unrelated['oofs_unrelated_{}'.format(i)] = oofs[i]['unrelated'].values
        disagreeds['oofs_disagreeds_{}'.format(i)] = oofs[i]['disagreed'].values
else:
    for i, oof in enumerate(oofs):
        agreeds['oofs_agreed_{}'.format(i)] = preds[i]['agreed'].values
        unrelated['oofs_unrelated_{}'.format(i)] = preds[i]['unrelated'].values
        disagreeds['oofs_disagreeds_{}'.format(i)] = preds[i]['disagreed'].values  

In [43]:
agreeds.corr()

,oofs_agreed_0,oofs_agreed_1,oofs_agreed_2,oofs_agreed_3,oofs_agreed_4,oofs_agreed_5,oofs_agreed_6,oofs_agreed_7,oofs_agreed_8,oofs_agreed_9,oofs_agreed_10,oofs_agreed_11,oofs_agreed_12,oofs_agreed_13,oofs_agreed_14,oofs_agreed_15,oofs_agreed_16,oofs_agreed_17,oofs_agreed_18
oofs_agreed_0,1.000000,0.985454,0.988323,0.986448,0.984380,0.987315,0.985907,0.973378,0.975002,0.977427,0.975342,0.976909,0.976050,0.976170,0.976952,0.976174,0.975576,0.975719,0.976823
oofs_agreed_1,0.985454,1.000000,0.988241,0.986295,0.995884,0.988644,0.985872,0.976543,0.978513,0.977921,0.978879,0.974114,0.978497,0.978545,0.977649,0.979854,0.979520,0.978561,0.979853
oofs_agreed_2,0.988323,0.988241,1.000000,0.987435,0.988193,0.996147,0.987102,0.970575,0.976039,0.976866,0.977199,0.973728,0.978022,0.977212,0.976467,0.977935,0.976717,0.977700,0.977855
oofs_agreed_3,0.986448,0.986295,0.987435,1.000000,0.984308,0.986038,0.996268,0.976276,0.979169,0.978624,0.983136,0.979642,0.983276,0.984887,0.978003,0.984594,0.979907,0.983532,0.985561
oofs_agreed_4,0.984380,0.995884,0.988193,0.984308,1.000000,0.988589,0.984010,0.974411,0.977064,0.976446,0.976906,0.971661,0.976735,0.976505,0.976157,0.977627,0.977743,0.976696,0.977415
oofs_agreed_5,0.987315,0.988644,0.996147,0.986038,0.988589,1.000000,0.986175,0.971035,0.974898,0.975453,0.976168,0.971536,0.976450,0.975633,0.974975,0.976913,0.975815,0.976077,0.976816
oofs_agreed_6,0.985907,0.985872,0.987102,0.996268,0.984010,0.986175,1.000000,0.975417,0.977783,0.977023,0.981811,0.977934,0.981879,0.983655,0.976346,0.983055,0.978560,0.982419,0.984168
oofs_agreed_7,0.973378,0.976543,0.970575,0.976276,0.974411,0.971035,0.975417,1.000000,0.988611,0.982953,0.986930,0.983626,0.979390,0.982267,0.982601,0.986865,0.989011,0.979822,0.983091
oofs_agreed_8,0.975002,0.978513,0.976039,0.979169,0.977064,0.974898,0.977783,0.988611,1.000000,0.987310,0.990456,0.983518,0.984780,0.985321,0.987066,0.990289,0.997975,0.984720,0.985544
oofs_agreed_9,0.977427,0.977921,0.976866,0.978624,0.976446,0.975453,0.977023,0.982953,0.987310,1.000000,0.984884,0.989911,0.989481,0.987805,0.998649,0.985065,0.987453,0.989079,0.988042


In [44]:
disagreeds.corr()

,oofs_disagreeds_0,oofs_disagreeds_1,oofs_disagreeds_2,oofs_disagreeds_3,oofs_disagreeds_4,oofs_disagreeds_5,oofs_disagreeds_6,oofs_disagreeds_7,oofs_disagreeds_8,oofs_disagreeds_9,oofs_disagreeds_10,oofs_disagreeds_11,oofs_disagreeds_12,oofs_disagreeds_13,oofs_disagreeds_14,oofs_disagreeds_15,oofs_disagreeds_16,oofs_disagreeds_17,oofs_disagreeds_18
oofs_disagreeds_0,1.000000,0.956195,0.965381,0.967006,0.970607,0.973446,0.969427,0.947563,0.953924,0.953973,0.953546,0.955650,0.952962,0.952829,0.948303,0.949407,0.948455,0.949762,0.947914
oofs_disagreeds_1,0.956195,1.000000,0.972663,0.969680,0.980875,0.961191,0.958854,0.940807,0.950593,0.941773,0.946255,0.942006,0.941758,0.944460,0.953687,0.960452,0.958069,0.955050,0.956554
oofs_disagreeds_2,0.965381,0.972663,1.000000,0.973263,0.974988,0.984404,0.967257,0.938211,0.951614,0.948799,0.949472,0.945175,0.947688,0.950114,0.955320,0.955833,0.955834,0.956771,0.954623
oofs_disagreeds_3,0.967006,0.969680,0.973263,1.000000,0.973872,0.970303,0.986330,0.955033,0.962220,0.955903,0.963340,0.957298,0.961192,0.963206,0.959059,0.969456,0.963931,0.967005,0.968558
oofs_disagreeds_4,0.970607,0.980875,0.974988,0.973872,1.000000,0.976251,0.972630,0.948467,0.960164,0.957989,0.960040,0.955228,0.957282,0.957418,0.956558,0.959968,0.958002,0.959996,0.956790
oofs_disagreeds_5,0.973446,0.961191,0.984404,0.970303,0.976251,1.000000,0.971418,0.938822,0.953536,0.955377,0.955229,0.950740,0.954774,0.956343,0.949685,0.951494,0.949397,0.953315,0.948816
oofs_disagreeds_6,0.969427,0.958854,0.967257,0.986330,0.972630,0.971418,1.000000,0.954179,0.960763,0.957241,0.964684,0.959270,0.963397,0.965154,0.951198,0.961519,0.955037,0.960886,0.960835
oofs_disagreeds_7,0.947563,0.940807,0.938211,0.955033,0.948467,0.938822,0.954179,1.000000,0.975049,0.961024,0.967412,0.970211,0.958116,0.962381,0.960166,0.969288,0.969771,0.959363,0.964948
oofs_disagreeds_8,0.953924,0.950593,0.951614,0.962220,0.960164,0.953536,0.960763,0.975049,1.000000,0.976337,0.977972,0.971753,0.971154,0.973719,0.972987,0.978265,0.990127,0.972071,0.971182
oofs_disagreeds_9,0.953973,0.941773,0.948799,0.955903,0.957989,0.955377,0.957241,0.961024,0.976337,1.000000,0.972073,0.978640,0.978412,0.978475,0.987397,0.966143,0.969127,0.976464,0.970646


In [45]:
unrelated.corr()

,oofs_unrelated_0,oofs_unrelated_1,oofs_unrelated_2,oofs_unrelated_3,oofs_unrelated_4,oofs_unrelated_5,oofs_unrelated_6,oofs_unrelated_7,oofs_unrelated_8,oofs_unrelated_9,oofs_unrelated_10,oofs_unrelated_11,oofs_unrelated_12,oofs_unrelated_13,oofs_unrelated_14,oofs_unrelated_15,oofs_unrelated_16,oofs_unrelated_17,oofs_unrelated_18
oofs_unrelated_0,1.000000,0.979483,0.982918,0.982116,0.980407,0.983841,0.982145,0.967810,0.969136,0.971615,0.969398,0.971206,0.969555,0.970029,0.970348,0.969500,0.968974,0.969428,0.970400
oofs_unrelated_1,0.979483,1.000000,0.985805,0.983235,0.993880,0.983194,0.981397,0.970652,0.972543,0.969819,0.972554,0.966131,0.969558,0.970829,0.973004,0.975851,0.975135,0.973627,0.975486
oofs_unrelated_2,0.982918,0.985805,1.000000,0.984631,0.985170,0.992641,0.983242,0.964436,0.969656,0.968727,0.970621,0.965539,0.969039,0.969442,0.971613,0.973276,0.971808,0.972618,0.973165
oofs_unrelated_3,0.982116,0.983235,0.984631,1.000000,0.981340,0.981809,0.994782,0.971740,0.974504,0.972338,0.978712,0.973816,0.977123,0.979705,0.973593,0.981349,0.975971,0.979870,0.982335
oofs_unrelated_4,0.980407,0.993880,0.985170,0.981340,1.000000,0.985285,0.980849,0.969350,0.972134,0.970553,0.971981,0.965403,0.970200,0.970659,0.971251,0.973084,0.973036,0.972064,0.972748
oofs_unrelated_5,0.983841,0.983194,0.992641,0.981809,0.985285,1.000000,0.982544,0.964530,0.968744,0.969367,0.970257,0.964712,0.970017,0.969717,0.968046,0.970220,0.969092,0.969949,0.970316
oofs_unrelated_6,0.982145,0.981397,0.983242,0.994782,0.980849,0.982544,1.000000,0.970761,0.973068,0.971318,0.977646,0.972614,0.976537,0.979104,0.970859,0.978754,0.973566,0.978102,0.979990
oofs_unrelated_7,0.967810,0.970652,0.964436,0.971740,0.969350,0.964530,0.970761,1.000000,0.986337,0.978792,0.983965,0.980390,0.974268,0.978164,0.978848,0.983940,0.986260,0.975830,0.979610
oofs_unrelated_8,0.969136,0.972543,0.969656,0.974504,0.972134,0.968744,0.973068,0.986337,1.000000,0.984056,0.987820,0.979701,0.980506,0.981747,0.983653,0.987454,0.996736,0.981314,0.982084
oofs_unrelated_9,0.971615,0.969819,0.968727,0.972338,0.970553,0.969367,0.971318,0.978792,0.984056,1.000000,0.981037,0.987146,0.986571,0.984970,0.996133,0.979866,0.982942,0.985613,0.983777


# Prepare Different Inputs

In [46]:
# Only use oofs
########### FOR RIDGE #############
ensemble_trains = trains.values
ensemble_tests = tests.values

# RidgeRegression

In [47]:
from sklearn.linear_model import RidgeClassifier
from sklearn import metrics

In [48]:
def get_prob(v):
    res = []
    for d in v:
        d = np.exp(d) / np.sum(np.exp(d))
        res.append(d)
    return np.array(res)

def fit_every_feature_model(feature_data, label, feature_test_data, fold_count=3, predict=True):
    predictions = np.zeros(shape=[len(feature_test_data), 3])
    fold_size = len(feature_data) // fold_count
    oofs = []
    
    log_loss = 0
    for fold_id in range(fold_count):
        print("Fold : ", fold_id)
        fold_start = fold_size * fold_id
        fold_end = fold_start + fold_size
        if fold_id == fold_count - 1:
            fold_end = len(feature_data)
                
        train_x = np.concatenate([feature_data[:fold_start], feature_data[fold_end:]])
        train_y = np.concatenate([label[:fold_start], label[fold_end:]])

        val_x = feature_data[fold_start:fold_end]
        val_y = label[fold_start:fold_end]        
        
        clf = RidgeClassifier().fit(train_x, train_y)
        print("Score", clf.score(val_x, val_y))
        
        if predict:
            prediction = get_prob(clf.decision_function(feature_test_data))
            oof_prediction = get_prob(clf.decision_function(val_x))

            score = metrics.log_loss(val_y, oof_prediction)
            print("Fold", fold_id, "log loss", score)
            log_loss += score
            oofs.append(oof_prediction)
            predictions += prediction        
        
    predictions /= fold_count   
    print("Training  Finish")

    return predictions, log_loss / fold_count, oofs

In [49]:
pred, log_loss, oofs = fit_every_feature_model(ensemble_trains, labels, ensemble_tests, fold_count=10)

Fold :  0
Score 0.8934643581344563
Fold 0 log loss 0.4350941487452475
Fold :  1
Score 0.8997660271408516
Fold 1 log loss 0.4150244164948867
Fold :  2
Score 0.8911558259241928
Fold 2 log loss 0.4347878648108567
Fold :  3
Score 0.8863203868351271
Fold 3 log loss 0.44601551541715645
Fold :  4
Score 0.900701918577445
Fold 4 log loss 0.4255465423154889
Fold :  5
Score 0.8869755108407424
Fold 5 log loss 0.44378103728851853
Fold :  6
Score 0.8762127593199189
Fold 6 log loss 0.4554272209406775
Fold :  7
Score 0.8882545624707534
Fold 7 log loss 0.4404644745130055
Fold :  8
Score 0.8920605209795663
Fold 8 log loss 0.43701455725662064
Fold :  9
Score 0.8943132545153945
Fold 9 log loss 0.43079371968839936
Training  Finish


In [50]:
def np_weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    weight_mask = np.max(weight_mask, axis=-1)
    norms = np.sum(weight_mask)
    
    y_true = np.argmax(y_true, axis=-1)
    y_pred = np.argmax(y_pred, axis=-1)
    
    res = ((y_true == y_pred) * weight_mask).sum() / norms
    return res

In [51]:
oofs = np.concatenate(oofs)
score = np_weighted_accuracy(to_categorical(labels), oofs)

In [52]:
print("score", score)
oofs_dir = "../data/p_ensemble/oofs/"
output_dir = "../data/p_ensemble/preds/"
onehot_pred_dir = "../data/p_ensemble/nn_one_hot/"

model_submit_prefix = "Ridge-Ensemble"

oofs_path = oofs_dir + model_submit_prefix
output_path = output_dir + model_submit_prefix
one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

print("Predicting training results...")
oofs = pd.DataFrame({"unrelated": oofs[:, 0], "agreed": oofs[:, 1], "disagreed": oofs[:, 2]})
submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
oofs.to_csv(submit_path, index=False)

test_predicts = pd.DataFrame({"unrelated": pred[:, 0], "agreed": pred[:, 1], "disagreed": pred[:, 2]})
submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
test_predicts.to_csv(submit_path, index=False) # 0.3343

print("Predicting labeled testing results...")
ids = pd.read_csv("../data/dataset/test.csv")
pred_labels = test_predicts.idxmax(axis=1)
sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
sub.to_csv(submit_path, index=False)

score 0.8690726824017866
Predicting training results...
Predicting labeled testing results...


## Scaled

In [53]:
def get_prob(v):
    res = []
    for d in v:
        d = np.exp(d) / np.sum(np.exp(d))
        res.append(d)
    return np.array(res)

def fit_every_feature_model(feature_data, label, feature_test_data, fold_count=3, predict=True):
    predictions = np.zeros(shape=[len(feature_test_data), 3])
    fold_size = len(feature_data) // fold_count
    oofs = []
    
    log_loss = 0
    for fold_id in range(fold_count):
        print("Fold : ", fold_id)
        fold_start = fold_size * fold_id
        fold_end = fold_start + fold_size
        if fold_id == fold_count - 1:
            fold_end = len(feature_data)
                
        train_x = np.concatenate([feature_data[:fold_start], feature_data[fold_end:]])
        train_y = np.concatenate([label[:fold_start], label[fold_end:]])

        val_x = feature_data[fold_start:fold_end]
        val_y = label[fold_start:fold_end]        
        
        clf = RidgeClassifier(class_weight={0: 1/16, 1:1/15, 2: 1/5}).fit(train_x, train_y)
        print("Score", clf.score(val_x, val_y))
        
        if predict:
            prediction = get_prob(clf.decision_function(feature_test_data))
            oof_prediction = get_prob(clf.decision_function(val_x))

            score = metrics.log_loss(val_y, oof_prediction)
            print("Fold", fold_id, "log loss", score)
            log_loss += score
            oofs.append(oof_prediction)
            predictions += prediction        
        
    predictions /= fold_count   
    print("Training  Finish")

    return predictions, log_loss / fold_count, oofs

In [54]:
pred, log_loss, oofs = fit_every_feature_model(ensemble_trains, labels, ensemble_tests, fold_count=10)

Fold :  0
Score 0.8881297769458742
Fold 0 log loss 0.4469899728250228
Fold :  1
Score 0.8964592107315551
Fold 1 log loss 0.42423624607529403
Fold :  2
Score 0.8879737950397754
Fold 2 log loss 0.4430657654748634
Fold :  3
Score 0.8800187178287319
Fold 3 log loss 0.45630396639771914
Fold :  4
Score 0.8979566370301045
Fold 4 log loss 0.4351545662364465
Fold :  5
Score 0.8824832319450944
Fold 5 log loss 0.4534675602155028
Fold :  6
Score 0.8762127593199189
Fold 6 log loss 0.46065279146871324
Fold :  7
Score 0.8822648572765559
Fold 7 log loss 0.4508315765795806
Fold :  8
Score 0.8888472937139292
Fold 8 log loss 0.44494612876787215
Fold :  9
Score 0.8913809776335901
Fold 9 log loss 0.43956186391668595
Training  Finish


In [55]:
oofs = np.concatenate(oofs)
score = np_weighted_accuracy(to_categorical(labels), oofs)

In [56]:
print("score", score)
oofs_dir = "../data/p_ensemble/oofs/"
output_dir = "../data/p_ensemble/preds/"
onehot_pred_dir = "../data/p_ensemble/nn_one_hot/"

model_submit_prefix = "RidgeScaled-Ensemble"

oofs_path = oofs_dir + model_submit_prefix
output_path = output_dir + model_submit_prefix
one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

print("Predicting training results...")
oofs = pd.DataFrame({"unrelated": oofs[:, 0], "agreed": oofs[:, 1], "disagreed": oofs[:, 2]})
submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
oofs.to_csv(submit_path, index=False)

test_predicts = pd.DataFrame({"unrelated": pred[:, 0], "agreed": pred[:, 1], "disagreed": pred[:, 2]})
submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
test_predicts.to_csv(submit_path, index=False) # 0.3343

print("Predicting labeled testing results...")
ids = pd.read_csv("../data/dataset/test.csv")
pred_labels = test_predicts.idxmax(axis=1)
sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
sub.to_csv(submit_path, index=False)

score 0.8777104350553753
Predicting training results...
Predicting labeled testing results...


# Logistic Ensembler

In [78]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, BayesianRidge, SGDClassifier

In [58]:
def get_prob(v):
    res = []
    for d in v:
        d = np.exp(d) / np.sum(np.exp(d))
        res.append(d)
    return np.array(res)

def fit_every_feature_model(feature_data, label, feature_test_data, fold_count=3, predict=True):
    predictions = np.zeros(shape=[len(feature_test_data), 3])
    fold_size = len(feature_data) // fold_count
    oofs = []
    
    log_loss = 0
    for fold_id in range(fold_count):
        print("Fold : ", fold_id)
        fold_start = fold_size * fold_id
        fold_end = fold_start + fold_size
        if fold_id == fold_count - 1:
            fold_end = len(feature_data)
                
        train_x = np.concatenate([feature_data[:fold_start], feature_data[fold_end:]])
        train_y = np.concatenate([label[:fold_start], label[fold_end:]])

        val_x = feature_data[fold_start:fold_end]
        val_y = label[fold_start:fold_end]        
        
        clf = LogisticRegression().fit(train_x, train_y)
        print("Score", clf.score(val_x, val_y))
        
        if predict:
            prediction = clf.predict_proba(feature_test_data)
            oof_prediction = clf.predict_proba(val_x)

            score = metrics.log_loss(val_y, oof_prediction)
            print("Fold", fold_id, "log loss", score)
            log_loss += score
            oofs.append(oof_prediction)
            predictions += prediction        
        
    predictions /= fold_count   
    print("Training  Finish")

    return predictions, log_loss / fold_count, oofs

In [59]:
pred, log_loss, oofs = fit_every_feature_model(ensemble_trains, labels, ensemble_tests, fold_count=10)

Fold :  0
Score 0.8930588051785993
Fold 0 log loss 0.26195867040356585
Fold :  1
Score 0.8994228669474341
Fold 1 log loss 0.24231697067161798
Fold :  2
Score 0.8911870223054126
Fold 2 log loss 0.2643755095049082
Fold :  3
Score 0.8871938855092809
Fold 3 log loss 0.2755187323987561
Fold :  4
Score 0.8991732958976759
Fold 4 log loss 0.24703062814133606
Fold :  5
Score 0.8874434565590391
Fold 5 log loss 0.2677488047703479
Fold :  6
Score 0.875214475120886
Fold 6 log loss 0.2942973136666923
Fold :  7
Score 0.8889096864763687
Fold 7 log loss 0.2674502654740498
Fold :  8
Score 0.8932771798471377
Fold 8 log loss 0.2611251260650516
Fold :  9
Score 0.8949995320834763
Fold 9 log loss 0.2569134369884683
Training  Finish


In [60]:
oofs = np.concatenate(oofs)
score = np_weighted_accuracy(to_categorical(labels), oofs)

In [61]:
print("score", score)
oofs_dir = "../data/p_ensemble/oofs/"
output_dir = "../data/p_ensemble/preds/"
onehot_pred_dir = "../data/p_ensemble/nn_one_hot/"

model_submit_prefix = "Logistic-Ensemble"

oofs_path = oofs_dir + model_submit_prefix
output_path = output_dir + model_submit_prefix
one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

print("Predicting training results...")
oofs = pd.DataFrame({"unrelated": oofs[:, 0], "agreed": oofs[:, 1], "disagreed": oofs[:, 2]})
submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
oofs.to_csv(submit_path, index=False)

test_predicts = pd.DataFrame({"unrelated": pred[:, 0], "agreed": pred[:, 1], "disagreed": pred[:, 2]})
submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
test_predicts.to_csv(submit_path, index=False) # 0.3343

print("Predicting labeled testing results...")
ids = pd.read_csv("../data/dataset/test.csv")
pred_labels = test_predicts.idxmax(axis=1)
sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
sub.to_csv(submit_path, index=False)

score 0.8706254369175598
Predicting training results...
Predicting labeled testing results...


## Scaled

In [62]:
def get_prob(v):
    res = []
    for d in v:
        d = np.exp(d) / np.sum(np.exp(d))
        res.append(d)
    return np.array(res)

def fit_every_feature_model(feature_data, label, feature_test_data, fold_count=3, predict=True):
    predictions = np.zeros(shape=[len(feature_test_data), 3])
    fold_size = len(feature_data) // fold_count
    oofs = []
    
    log_loss = 0
    for fold_id in range(fold_count):
        print("Fold : ", fold_id)
        fold_start = fold_size * fold_id
        fold_end = fold_start + fold_size
        if fold_id == fold_count - 1:
            fold_end = len(feature_data)
                
        train_x = np.concatenate([feature_data[:fold_start], feature_data[fold_end:]])
        train_y = np.concatenate([label[:fold_start], label[fold_end:]])

        val_x = feature_data[fold_start:fold_end]
        val_y = label[fold_start:fold_end]        
        
        clf = LogisticRegression(class_weight={0: 1/16, 1:1/15, 2: 1/5}).fit(train_x, train_y)
        print("Score", clf.score(val_x, val_y))
        
        if predict:
            prediction = clf.predict_proba(feature_test_data)
            oof_prediction = clf.predict_proba(val_x)

            score = metrics.log_loss(val_y, oof_prediction)
            print("Fold", fold_id, "log loss", score)
            log_loss += score
            oofs.append(oof_prediction)
            predictions += prediction        
        
    predictions /= fold_count   
    print("Training  Finish")

    return predictions, log_loss / fold_count, oofs

In [63]:
pred, log_loss, oofs = fit_every_feature_model(ensemble_trains, labels, ensemble_tests, fold_count=10)

Fold :  0
Score 0.8905007019185774
Fold 0 log loss 0.3387896340363898
Fold :  1
Score 0.8987053501793791
Fold 1 log loss 0.3108030978317047
Fold :  2
Score 0.8901575417251599
Fold 2 log loss 0.3331646136281357
Fold :  3
Score 0.8842614256746217
Fold 3 log loss 0.345689462857064
Fold :  4
Score 0.8987989393230386
Fold 4 log loss 0.30834979744613983
Fold :  5
Score 0.8858212447356106
Fold 5 log loss 0.33644688432046016
Fold :  6
Score 0.8763063484635782
Fold 6 log loss 0.36819509793620286
Fold :  7
Score 0.8869131180783029
Fold 7 log loss 0.3377457218367191
Fold :  8
Score 0.8923100920293247
Fold 8 log loss 0.3276271239778752
Fold :  9
Score 0.8944692266899585
Fold 9 log loss 0.3214368732406488
Training  Finish


In [64]:
oofs = np.concatenate(oofs)
score = np_weighted_accuracy(to_categorical(labels), oofs)

In [65]:
print("score", score)
oofs_dir = "../data/p_ensemble/oofs/"
output_dir = "../data/p_ensemble/preds/"
onehot_pred_dir = "../data/p_ensemble/nn_one_hot/"

model_submit_prefix = "LogisticScaled-Ensemble"

oofs_path = oofs_dir + model_submit_prefix
output_path = output_dir + model_submit_prefix
one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

print("Predicting training results...")
oofs = pd.DataFrame({"unrelated": oofs[:, 0], "agreed": oofs[:, 1], "disagreed": oofs[:, 2]})
submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
oofs.to_csv(submit_path, index=False)

test_predicts = pd.DataFrame({"unrelated": pred[:, 0], "agreed": pred[:, 1], "disagreed": pred[:, 2]})
submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
test_predicts.to_csv(submit_path, index=False) # 0.3343

print("Predicting labeled testing results...")
ids = pd.read_csv("../data/dataset/test.csv")
pred_labels = test_predicts.idxmax(axis=1)
sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
sub.to_csv(submit_path, index=False)

score 0.8750042278447884
Predicting training results...
Predicting labeled testing results...
